## **02 — Preprocess and merge fitness datasets:** (Brochado + Cacace → unified fitness table)

This notebook loads the cleaned Brochado and Cacace fitness tables (stage1), checks whether numeric normalization is needed, harmonizes drug identifiers (3-letter codes, drug names, inchikeys), merges both datasets on `inchikey`, selects a fixed set of strain fitness columns, rounds fitness values, and exports a unified fitness table for strain-space construction.


### Inputs
- `feature_pipeline/strain_space/inputs/stage1/raw_brochado_fitness.csv`
- `feature_pipeline/strain_space/inputs/stage1/raw_cacace_fitness.csv`
- `data/reference/drug_lists/list_antibacterial.csv`
- `data/reference/drug_lists/3_letter_code_cacace.csv`

### Output
- `feature_pipeline/strain_space/inputs/stage1/raw_fitness.csv`

### Notes / assumptions
- Fitness values from both datasets are assumed comparable on the same numeric scale (no additional normalization applied).
- Cacace drug identifiers are mapped from `3_letter_code → drug → inchikey` using reference CSVs.
- Brochado drug names are mapped directly `drug → inchikey` using `list_antibacterial.csv`.
- Merge is performed with an **outer join** on `inchikey`, then rows missing `inchikey` are dropped.
- The output retains a fixed list of strain columns; all values are coerced to numeric and rounded to 2 decimals.


In [1]:
import pandas as pd
from halo.paths import FEATURE_PIPELINE, DRUG_LISTS

In [2]:
c_fitness = pd.read_csv(FEATURE_PIPELINE / "strain_space" / "inputs" / "stage1" / "raw_cacace_fitness.csv").copy()
b_fitness = pd.read_csv(FEATURE_PIPELINE / "strain_space" / "inputs" / "stage1" / "raw_brochado_fitness.csv").copy()

list_antibacterial = pd.read_csv(DRUG_LISTS / "list_antibacterial.csv")
codes_cacace = pd.read_csv(DRUG_LISTS / "3_letter_code_cacace.csv")

In [3]:
codes_cacace.head()

,drug,3_letter_code
0,ADEP,ADEP
1,Amoxicillinclavulanic,AMXCLA
2,Amoxicillin,AMX
3,Acetylsalicylicacid,ASA
4,Acetylsalicylicacid,ASA


In [4]:
list_antibacterial.head()

,drug,inchikey
0,a22,LZTCFLDZLBOLDW-UHFFFAOYSA-N
1,acetylsalicylic acid,BSYNRYMUTXBXSQ-UHFFFAOYSA-N
2,acetylsalicylicacid,BSYNRYMUTXBXSQ-UHFFFAOYSA-N
3,acetylsalisylic acid,BSYNRYMUTXBXSQ-UHFFFAOYSA-N
4,alahopcin,NTBVVEFUJUCXPF-FYCPLRARSA-N


In [5]:
c_fitness.head()

,drug,bacillus subtilis,staphylococcus aureus dsm 20231,staphylococcus aureus newman,streptococcus pneumoniae
0,5FU,NaN,0.741594,NaN,NaN
1,ADEP,NaN,0.717285,0.563376,NaN
2,ALF,NaN,0.985689,NaN,NaN
3,ALL,NaN,0.996244,NaN,NaN
4,AMX,0.328294,0.370839,0.866437,0.930972


In [10]:
c_fitness = c_fitness.rename(columns={'drug': '3_letter_code'})
c_fitness.head() # cacace

,3_letter_code,bacillus subtilis,staphylococcus aureus dsm 20231,staphylococcus aureus newman,streptococcus pneumoniae
0,5FU,NaN,0.741594,NaN,NaN
1,ADEP,NaN,0.717285,0.563376,NaN
2,ALF,NaN,0.985689,NaN,NaN
3,ALL,NaN,0.996244,NaN,NaN
4,AMX,0.328294,0.370839,0.866437,0.930972


In [11]:
b_fitness.head() # brochado

,drug,3_letter_code,escherichia coli bw25113,escherichia coli iai1,salmonella typhimurium lt2,salmonella typhimurium 14028,pseudomonas aeruginosa pao1,pseudomonas aeruginosa pa14
0,Amoxicillin,AMX,0.45,0.00,0.84,0.76,0.89,0.92
1,Oxacillin,OXA,0.78,0.83,0.85,0.86,0.88,0.93
2,Cefotaxime,CTX,0.57,0.39,0.71,0.79,0.00,0.76
3,Cefaclor,CEC,0.47,0.55,0.00,0.00,0.90,0.92
4,Cefsulodin,CFS,0.62,0.86,0.82,0.89,0.00,0.00


check if the data needs normalization within df before merging:

In [12]:
b_fitness.iloc[:, 2:].describe()

,escherichia coli bw25113,escherichia coli iai1,salmonella typhimurium lt2,salmonella typhimurium 14028,pseudomonas aeruginosa pao1,pseudomonas aeruginosa pa14
count,78.000000,78.000000,78.000000,78.000000,76.000000,76.000000
mean,0.544231,0.542821,0.631026,0.644487,0.612632,0.598421
std,0.312225,0.350402,0.307607,0.301919,0.363976,0.376804
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.340000,0.165000,0.535000,0.515000,0.357500,0.060000
50%,0.575000,0.665000,0.735000,0.770000,0.780000,0.820000
75%,0.825000,0.857500,0.850000,0.860000,0.872500,0.880000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
c_fitness.iloc[:, 1:].describe()

,bacillus subtilis,staphylococcus aureus dsm 20231,staphylococcus aureus newman,streptococcus pneumoniae
count,65.000000,121.000000,66.000000,62.000000
mean,0.660442,0.762736,0.678472,0.728084
std,0.238551,0.239221,0.252357,0.229477
min,0.060193,0.046053,0.046897,0.238543
25%,0.434687,0.629780,0.454930,0.525810
50%,0.689389,0.833288,0.709976,0.800313
75%,0.902587,0.964438,0.931146,0.932661
max,0.996811,1.000000,0.995899,0.990781


comparison:  
* Both datasets share the same numeric range (0–1).  
* Cacace has a narrower dynamic range and slightly higher baseline fitness, but not by a scale factor.     
* Differences reflect biological variation (different strains, different assay sensitivity), not scaling bias.  
-> No numeric normalization is needed.

### cleaning up names, mapping inchikeys:

In [14]:
b_fitness['drug'] = b_fitness['drug'].astype(str).str.strip().str.lower()
b_fitness['3_letter_code'] = b_fitness['3_letter_code'].astype(str).str.strip().str.lower()
c_fitness['3_letter_code'] = c_fitness['3_letter_code'].astype(str).str.strip().str.lower()
list_antibacterial['drug'] = list_antibacterial['drug'].astype(str).str.strip().str.lower()
list_antibacterial['inchikey'] = list_antibacterial['inchikey'].astype(str).str.strip().str.lower()
codes_cacace['drug'] = codes_cacace['drug'].astype(str).str.strip().str.lower()
codes_cacace['3_letter_code'] = codes_cacace['3_letter_code'].astype(str).str.strip().str.lower()

In [15]:
codes_cacace = codes_cacace.drop_duplicates(subset=['drug', '3_letter_code'])

In [16]:
look_up_name = dict(zip(codes_cacace['3_letter_code'], codes_cacace['drug']))
c_fitness['drug'] = c_fitness['3_letter_code'].map(look_up_name)

In [17]:
c_fitness.head()

,3_letter_code,bacillus subtilis,staphylococcus aureus dsm 20231,staphylococcus aureus newman,streptococcus pneumoniae,drug
0,5fu,NaN,0.741594,NaN,NaN,fluorouracil
1,adep,NaN,0.717285,0.563376,NaN,adep
2,alf,NaN,0.985689,NaN,NaN,alfacalcidol
3,all,NaN,0.996244,NaN,NaN,NaN
4,amx,0.328294,0.370839,0.866437,0.930972,amoxicillin


In [18]:
c_fitness['drug'].isna().sum()

np.int64(15)

In [16]:
# c_fitness[c_fitness['drug'].isna()]

In [17]:
len(c_fitness)

121

In [18]:
look_up_inchikeys = dict(zip(list_antibacterial['drug'], list_antibacterial['inchikey']))
c_fitness['inchikey'] = c_fitness['drug'].map(look_up_inchikeys)
c_fitness.head()

,3_letter_code,bacillus subtilis,staphylococcus aureus dsm 20231,staphylococcus aureus newman,streptococcus pneumoniae,drug,inchikey
0,5fu,NaN,0.741594,NaN,NaN,fluorouracil,ghasvsinzrgabv-uhfffaoysa-n
1,adep,NaN,0.717285,0.563376,NaN,adep,NaN
2,alf,NaN,0.985689,NaN,NaN,alfacalcidol,NaN
3,all,NaN,0.996244,NaN,NaN,NaN,NaN
4,amx,0.328294,0.370839,0.866437,0.930972,amoxicillin,lsqzjlsuydqpkj-njbdsqktsa-n


In [19]:
c_fitness['inchikey'].isna().sum()

np.int64(43)

In [20]:
b_fitness['inchikey'] = b_fitness['drug'].map(look_up_inchikeys)
b_fitness.head()

,drug,3_letter_code,escherichia coli bw25113,escherichia coli iai1,salmonella typhimurium lt2,salmonella typhimurium 14028,pseudomonas aeruginosa pao1,pseudomonas aeruginosa pa14,inchikey
0,amoxicillin,amx,0.45,0.00,0.84,0.76,0.89,0.92,lsqzjlsuydqpkj-njbdsqktsa-n
1,oxacillin,oxa,0.78,0.83,0.85,0.86,0.88,0.93,uwyhmgvutgawsp-jkifevaisa-n
2,cefotaxime,ctx,0.57,0.39,0.71,0.79,0.00,0.76,gprbekhldvquje-qswimtsfsa-n
3,cefaclor,cec,0.47,0.55,0.00,0.00,0.90,0.92,qyiyflotgylrgg-gpccphfnsa-n
4,cefsulodin,cfs,0.62,0.86,0.82,0.89,0.00,0.00,sylkglmblaagsc-qlvmhmetsa-n


In [21]:
b_fitness['inchikey'].isna().sum()

np.int64(9)

In [22]:
merged = b_fitness.merge(c_fitness, on='inchikey', how='outer')
merged['drug'] = merged['drug_x'].fillna(merged['drug_y'])
merged['3_letter_code'] = merged['3_letter_code_x'].fillna(merged['3_letter_code_y'])
merged = merged.drop(columns=['drug_x', 'drug_y', '3_letter_code_x', '3_letter_code_y'])

name_cols = ['drug', '3_letter_code', 'inchikey']
fitness_cols = ['escherichia coli bw25113', 'escherichia coli iai1', 
                'salmonella typhimurium lt2', 'salmonella typhimurium 14028',
                'pseudomonas aeruginosa pao1', 'pseudomonas aeruginosa pa14',
                'bacillus subtilis', 'staphylococcus aureus dsm 20231', 'staphylococcus aureus newman',
                'streptococcus pneumoniae']

merged[fitness_cols] = merged[fitness_cols].apply(pd.to_numeric, errors='coerce')
merged[fitness_cols] = merged[fitness_cols].round(2)

df = merged[name_cols + fitness_cols]
df.head()

,drug,3_letter_code,inchikey,escherichia coli bw25113,escherichia coli iai1,salmonella typhimurium lt2,salmonella typhimurium 14028,pseudomonas aeruginosa pao1,pseudomonas aeruginosa pa14,bacillus subtilis,staphylococcus aureus dsm 20231,staphylococcus aureus newman,streptococcus pneumoniae
0,spiramycin,spm,actoxuheucptew-ceuobaopsa-n,0.73,0.79,0.80,0.84,0.86,0.88,NaN,NaN,NaN,NaN
1,clarithromycin,clr,agoydepgaoxock-kcbohyoisa-n,0.45,0.57,0.60,0.54,0.36,0.44,0.40,0.47,0.39,0.34
2,doxorubicin,dxr,aojjsuzboxzqnb-tzssrymlsa-n,0.81,0.86,0.81,0.84,0.90,0.91,0.85,0.92,0.82,0.92
3,auranofin,aur,aujrcfubupvwsz-xtzhgvarsa-m,NaN,NaN,NaN,NaN,NaN,NaN,0.32,0.37,0.66,0.31
4,teicoplanin,tec,bjnllbuohpvgft-cayrisatsa-n,0.88,0.88,0.85,0.87,0.82,0.83,0.76,0.99,0.87,0.34


In [23]:
df['drug'].isna().sum()

np.int64(0)

In [24]:
df['3_letter_code'].isna().sum()

np.int64(0)

In [25]:
df['inchikey'].isna().sum()

np.int64(387)

In [26]:
df = df.dropna(subset=['inchikey'])

In [27]:
df.shape

(103, 13)

In [28]:
out_path = FEATURE_PIPELINE / "strain_space" / "inputs" / "stage1" / "raw_fitness.csv"
df.to_csv(out_path, index=False)